# Step 4: Select one sample per hour for the stats calculation done in step 3

Do notice that, usually the satellite offers a full scan of the continent in an interval of 15 minutes. But, the weather station collects data hourly. Therefore, it would be the best for us to select the sample according to the weather station data collection schedule.

In [1]:
import os
import pandas as pd
import scipy
import numpy as np

### TO-DO:

Change the directory for the input file if needed.

In [2]:
os.getcwd()

## TO-DO: Change the directory if needed
os.chdir("/srv/scratch/NOAA/GOES_Hourly_Statistics/stats_result/")
os.getcwd()

'/srv/scratch/NOAA/GOES_Hourly_Statistics/stats_result'

In [3]:
lake = pd.read_csv('03_opencv_lat_lon_14f15s_lake_0.csv', dtype={'Date': str, 'Time': str})
lake

,Date,Time,Mean,Centroid_lon,Centroid_lat,Std_lon,Std_lat,Skewness_lon,Skewness_lat,Kurtosis_lon,Kurtosis_lat,Cloud_Cover_Count,Total_Lake_Pixel,Percent_Cloud_Cover
0,20141001,0000,0.001928,-86.746619,43.883567,0.662943,1.224128,0.649933,0.080196,2.899997,1.760560,1364,2458,0.554923
1,20141001,0030,0.001662,-86.764055,43.894928,0.625222,1.220894,0.650172,0.021298,2.968796,1.716812,1125,2271,0.495376
2,20141001,0100,0.001935,-86.742887,43.944112,0.643303,1.220939,0.645693,0.036302,2.965771,1.725964,1331,2411,0.552053
3,20141001,0115,0.002064,-86.747547,43.863015,0.649044,1.225484,0.636756,0.125066,2.966074,1.736784,1390,2452,0.566884
4,20141001,0130,0.002187,-86.736641,43.907192,0.649674,1.230919,0.616406,0.057302,2.865603,1.733103,1301,2359,0.551505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15594,20150331,2245,0.286596,-86.550300,44.350345,0.676449,1.228545,0.321565,-0.496684,2.466423,1.893066,2098,3599,0.582940
15595,20150331,2300,0.237422,-86.562227,44.344671,0.668177,1.209333,0.345187,-0.488859,2.506484,1.915377,2216,3599,0.615727
15596,20150331,2315,0.191543,-86.598340,44.278719,0.660563,1.215461,0.407725,-0.419515,2.598452,1.850461,2454,3599,0.681856
15597,20150331,2330,0.135385,-86.622115,44.248200,0.652580,1.209073,0.430544,-0.366463,2.653697,1.823360,3031,3599,0.842178


In [4]:
# Add a ' to reduce error in pandas reading the Time elements with leading 0.

lake['Time'] = lake['Time'].apply(lambda x:"'"+x)

In [5]:
h_list = []
m_list = []
for t in lake['Time']:
    h_list.append(t[1:3])
    m_list.append(t[-2:])
lake['Hour'] = h_list
lake['Min'] = m_list
lake

,Date,Time,Mean,Centroid_lon,Centroid_lat,Std_lon,Std_lat,Skewness_lon,Skewness_lat,Kurtosis_lon,Kurtosis_lat,Cloud_Cover_Count,Total_Lake_Pixel,Percent_Cloud_Cover,Hour,Min
0,20141001,'0000,0.001928,-86.746619,43.883567,0.662943,1.224128,0.649933,0.080196,2.899997,1.760560,1364,2458,0.554923,00,00
1,20141001,'0030,0.001662,-86.764055,43.894928,0.625222,1.220894,0.650172,0.021298,2.968796,1.716812,1125,2271,0.495376,00,30
2,20141001,'0100,0.001935,-86.742887,43.944112,0.643303,1.220939,0.645693,0.036302,2.965771,1.725964,1331,2411,0.552053,01,00
3,20141001,'0115,0.002064,-86.747547,43.863015,0.649044,1.225484,0.636756,0.125066,2.966074,1.736784,1390,2452,0.566884,01,15
4,20141001,'0130,0.002187,-86.736641,43.907192,0.649674,1.230919,0.616406,0.057302,2.865603,1.733103,1301,2359,0.551505,01,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15594,20150331,'2245,0.286596,-86.550300,44.350345,0.676449,1.228545,0.321565,-0.496684,2.466423,1.893066,2098,3599,0.582940,22,45
15595,20150331,'2300,0.237422,-86.562227,44.344671,0.668177,1.209333,0.345187,-0.488859,2.506484,1.915377,2216,3599,0.615727,23,00
15596,20150331,'2315,0.191543,-86.598340,44.278719,0.660563,1.215461,0.407725,-0.419515,2.598452,1.850461,2454,3599,0.681856,23,15
15597,20150331,'2330,0.135385,-86.622115,44.248200,0.652580,1.209073,0.430544,-0.366463,2.653697,1.823360,3031,3599,0.842178,23,30


In [6]:
lake[lake['Min'] == '00'] #3103

,Date,Time,Mean,Centroid_lon,Centroid_lat,Std_lon,Std_lat,Skewness_lon,Skewness_lat,Kurtosis_lon,Kurtosis_lat,Cloud_Cover_Count,Total_Lake_Pixel,Percent_Cloud_Cover,Hour,Min
0,20141001,'0000,0.001928,-86.746619,43.883567,0.662943,1.224128,0.649933,0.080196,2.899997,1.760560,1364,2458,0.554923,00,00
2,20141001,'0100,0.001935,-86.742887,43.944112,0.643303,1.220939,0.645693,0.036302,2.965771,1.725964,1331,2411,0.552053,01,00
6,20141001,'0200,0.001960,-86.748614,43.942191,0.632510,1.227168,0.676957,0.002681,3.004695,1.741192,1278,2373,0.538559,02,00
10,20141001,'0300,0.001980,-86.727879,43.950843,0.643921,1.217663,0.555008,0.031485,2.822773,1.688957,1281,2324,0.551205,03,00
13,20141001,'0400,0.001894,-86.755259,43.912042,0.639526,1.217441,0.689993,0.026102,3.043969,1.729670,1270,2374,0.534962,04,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15581,20150331,'1900,0.424953,-86.452618,44.227281,0.634857,1.384043,0.424670,-0.303650,2.832995,1.487664,1990,3599,0.552931,19,00
15585,20150331,'2000,0.483508,-86.392889,44.431771,0.642376,1.309728,0.133097,-0.637916,2.567251,1.903678,1844,3599,0.512365,20,00
15588,20150331,'2100,0.476928,-86.429517,44.437452,0.687164,1.252320,0.112368,-0.644982,2.347145,2.093536,1800,3599,0.500139,21,00
15591,20150331,'2200,0.389742,-86.505643,44.398012,0.699705,1.242613,0.215426,-0.534133,2.299630,1.938475,1888,3599,0.524590,22,00


In [7]:
len(lake['Date'].unique())

182

In [8]:
list_da = []
list_ti = []
list_m = []
list_cx = []
list_cy = []
list_sx = []
list_sy = []
list_skx = []
list_sky = []
list_kx = []
list_ky = []
list_cloud_pixel_num = []
list_non_nan_num = []
list_percentage_cloud_cover = []
list_len =[]
list_case = []

In [9]:
# a = lake[(lake['Date'] == 20171231) & (lake['Hour'] == '23')& (lake['Min'] == '45')]
# a

In [10]:
# a['Date'].values[0]

In [11]:
# if ((len(a) == 1) & ((a['Min']== '45').bool())):
#     print('exist')
# else:
#     print('no-exist')
        

In [12]:
# lake = lake.iloc[2:6, :]
# lake

In [13]:
for d in lake['Date'].unique():
#     print(d)
    for t1 in (lake[lake['Date'] == d]['Hour']).unique():
        t1_v = lake[(lake['Date'] == d) & (lake['Hour'] == t1)]
        t1_n = len(lake[(lake['Date'] == d) & (lake['Hour'] == t1)])
        n00 = len(lake[(lake['Date'] == d) & (lake['Hour'] == t1)& (lake['Min'] == '00')])
        n15 = len(lake[(lake['Date'] == d) & (lake['Hour'] == t1)& (lake['Min'] == '15')])
        n30 = len(lake[(lake['Date'] == d) & (lake['Hour'] == t1)& (lake['Min'] == '30')])
        n45 = len(lake[(lake['Date'] == d) & (lake['Hour'] == t1)& (lake['Min'] == '45')])  
#         print(n00)
        list_len.append(t1_n)
        found = 0 
        while found < 1:
            if n00 == 1:
                row_f = lake[(lake['Date'] == d) & (lake['Hour'] == t1)& (lake['Min'] == '00')]
                list_da.append(row_f['Date'].values[0])
                list_ti.append(row_f['Time'].values[0]) 
                list_m.append(row_f['Mean'].values[0])
                
                list_cx.append(row_f['Centroid_lon'].values[0])
                list_cy.append(row_f['Centroid_lat'].values[0])
                list_sx.append(row_f['Std_lon'].values[0])
                list_sy.append(row_f['Std_lat'].values[0])
                list_skx.append(row_f['Skewness_lon'].values[0])
                list_sky.append(row_f['Skewness_lat'].values[0])
                list_kx.append(row_f['Kurtosis_lon'].values[0])
                list_ky.append(row_f['Kurtosis_lat'].values[0])
                list_cloud_pixel_num.append(row_f['Cloud_Cover_Count'].values[0])
                list_non_nan_num.append(row_f['Total_Lake_Pixel'].values[0])
                list_percentage_cloud_cover.append(row_f['Percent_Cloud_Cover'].values[0])
                list_case.append('s00')
                print('1---'+ str(row_f['Date'].values[0])+ row_f['Time'].values[0])
                found = found +1
                print(found)
            else:
                if ((n00 == 0) & (n15 == 0) & (n45 == 0)):
                    list_da.append(t1_v['Date'].values[0])
                    list_ti.append('N/A')
                    list_m.append('N/A')
                    
                    list_cx.append('N/A')
                    list_cy.append('N/A')
                    list_sx.append('N/A')
                    list_sy.append('N/A')
                    list_skx.append('N/A')
                    list_sky.append('N/A')
                    list_kx.append('N/A')
                    list_ky.append('N/A')
                    list_cloud_pixel_num.append('N/A')
                    list_non_nan_num.append('N/A')
                    list_percentage_cloud_cover.append('N/A')
                    list_case.append('s30')
                    print('2---'+ str(row_f['Date'].values[0]) + row_f['Time'].values[0])
                    found = found +1
                elif((n00 == 0) & (n15 == 1) & (n45 == 0)):
                    row_f = lake[(lake['Date'] == d) & (lake['Hour'] == t1)& (lake['Min'] == '15')]
                    list_da.append(row_f['Date'].values[0])
                    list_ti.append(row_f['Time'].values[0])
                    list_m.append(row_f['Mean'].values[0])
                    list_cx.append(row_f['Centroid_lon'].values[0])
                    list_cy.append(row_f['Centroid_lat'].values[0])
                    list_sx.append(row_f['Std_lon'].values[0])
                    list_sy.append(row_f['Std_lat'].values[0])
                    list_skx.append(row_f['Skewness_lon'].values[0])
                    list_sky.append(row_f['Skewness_lat'].values[0])
                    list_kx.append(row_f['Kurtosis_lon'].values[0])
                    list_ky.append(row_f['Kurtosis_lat'].values[0])
                    list_cloud_pixel_num.append(row_f['Cloud_Cover_Count'].values[0])
                    list_non_nan_num.append(row_f['Total_Lake_Pixel'].values[0])
                    list_percentage_cloud_cover.append(row_f['Percent_Cloud_Cover'].values[0])
                    list_case.append('s15')
                    print('3---' + str(row_f['Date'].values[0])+ row_f['Time'].values[0])
                    found = found +1
                elif((n00 == 0) & (n15 == 0) & (n45 == 1)):
                    row_f = lake[(lake['Date'] == d) & (lake['Hour'] == t1)& (lake['Min'] == '45')]
                    list_da.append(row_f['Date'].values[0])
                    list_ti.append(row_f['Time'].values[0])
                    list_m.append(row_f['Mean'].values[0])
                    list_cx.append(row_f['Centroid_lon'].values[0])
                    list_cy.append(row_f['Centroid_lat'].values[0])
                    list_sx.append(row_f['Std_lon'].values[0])
                    list_sy.append(row_f['Std_lat'].values[0])
                    list_skx.append(row_f['Skewness_lon'].values[0])
                    list_sky.append(row_f['Skewness_lat'].values[0])
                    list_kx.append(row_f['Kurtosis_lon'].values[0])
                    list_ky.append(row_f['Kurtosis_lat'].values[0])
                    list_cloud_pixel_num.append(row_f['Cloud_Cover_Count'].values[0])
                    list_non_nan_num.append(row_f['Total_Lake_Pixel'].values[0])
                    list_percentage_cloud_cover.append(row_f['Percent_Cloud_Cover'].values[0])
                    list_case.append('s45')
                    print('4---' + str(row_f['Date'].values[0])+ row_f['Time'].values[0])
                    found = found +1
                else:
                    row_f = lake[(lake['Date'] == d) & (lake['Hour'] == t1)& (lake['Min'] == '45')]
                    row_f_2 = lake[(lake['Date'] == d) & (lake['Hour'] == t1)& (lake['Min'] == '15')]
                    print(row_f)
                    list_da.append(row_f['Date'].values[0])
                    list_ti.append(row_f['Time'].values[0])
                    list_m.append(((row_f['Mean'].values[0]) + (row_f_2['Mean'].values[0]))/2)
                    
                    list_cx.append(((row_f['Centroid_lon'].values[0]) + (row_f_2['Centroid_lon'].values[0]))/2)
                    list_cy.append(((row_f['Centroid_lat'].values[0]) + (row_f_2['Centroid_lat'].values[0]))/2)
                    list_sx.append(((row_f['Std_lon'].values[0]) + (row_f_2['Std_lon'].values[0]))/2)
                    list_sy.append(((row_f['Std_lat'].values[0]) + (row_f_2['Std_lat'].values[0]))/2)
                    list_skx.append(((row_f['Skewness_lon'].values[0])+ (row_f_2['Skewness_lon'].values[0]))/2)
                    list_sky.append(((row_f['Skewness_lat'].values[0])+ (row_f_2['Skewness_lat'].values[0]))/2)
                    list_kx.append(((row_f['Kurtosis_lon'].values[0])+(row_f_2['Kurtosis_lon'].values[0]))/2)
                    list_ky.append(((row_f['Kurtosis_lat'].values[0])+ (row_f_2['Kurtosis_lat'].values[0]))/2)
                    list_cloud_pixel_num.append(((row_f['Cloud_Cover_Count'].values[0])+ (row_f_2['Cloud_Cover_Count'].values[0]))/2)
                    list_non_nan_num.append(((row_f['Total_Lake_Pixel'].values[0])+ (row_f_2['Total_Lake_Pixel'].values[0]))/2)
                    list_percentage_cloud_cover.append(((row_f['Percent_Cloud_Cover'].values[0])+ (row_f_2['Percent_Cloud_Cover'].values[0]))/2)

                    list_case.append('s1545')
                    print('5---' + str(row_f['Date'].values[0])+ row_f['Time'].values[0])
                    found = found +1
                  
                  
                  
                  
        



1---20141001'0000
1
1---20141001'0100
1
1---20141001'0200
1
1---20141001'0300
1
1---20141001'0400
1
1---20141001'0500
1
1---20141001'0600
1
1---20141001'0700
1
1---20141001'0800
1
1---20141001'0900
1
1---20141001'1000
1
1---20141001'1100
1
1---20141001'1200
1
1---20141001'1300
1
1---20141001'1400
1
1---20141001'1500
1
1---20141001'1600
1
1---20141001'1700
1
1---20141001'1800
1
1---20141001'1900
1
1---20141001'2000
1
1---20141001'2100
1
1---20141001'2200
1
1---20141001'2300
1
1---20141002'0000
1
1---20141002'0100
1
1---20141002'0200
1
1---20141002'0300
1
1---20141002'0400
1
1---20141002'0500
1
1---20141002'0600
1
1---20141002'0700
1
1---20141002'0800
1
1---20141002'0900
1
1---20141002'1000
1
1---20141002'1100
1
1---20141002'1200
1
1---20141002'1300
1
1---20141002'1400
1
1---20141002'1500
1
1---20141002'1600
1
1---20141002'1700
1
1---20141002'1800
1
1---20141002'1900
1
3---20141002'2015
1---20141002'2100
1
1---20141002'2200
1
1---20141002'2300
1
1---20141003'0000
1
1---20141003'0100
1
1-

1---20141018'1200
1
1---20141018'1300
1
1---20141018'1400
1
1---20141018'1500
1
1---20141018'1600
1
1---20141018'1700
1
1---20141018'1800
1
1---20141018'1900
1
1---20141018'2000
1
1---20141018'2100
1
1---20141018'2200
1
1---20141018'2300
1
1---20141019'0000
1
1---20141019'0100
1
1---20141019'0200
1
1---20141019'0300
1
1---20141019'0400
1
1---20141019'0500
1
1---20141019'0600
1
1---20141019'0700
1
1---20141019'0800
1
1---20141019'0900
1
1---20141019'1000
1
1---20141019'1100
1
1---20141019'1200
1
1---20141019'1300
1
1---20141019'1400
1
1---20141019'1500
1
1---20141019'1600
1
1---20141019'1700
1
1---20141019'1800
1
1---20141019'1900
1
1---20141019'2000
1
1---20141019'2100
1
1---20141019'2200
1
1---20141019'2300
1
1---20141020'0000
1
1---20141020'0100
1
1---20141020'0200
1
1---20141020'0300
1
1---20141020'0400
1
1---20141020'0500
1
1---20141020'0600
1
1---20141020'0700
1
1---20141020'0800
1
1---20141020'0900
1
1---20141020'1000
1
1---20141020'1100
1
1---20141020'1200
1
1---20141020'1300
1


1---20141104'2200
1
1---20141104'2300
1
1---20141105'0000
1
1---20141105'0100
1
1---20141105'0200
1
1---20141105'0300
1
1---20141105'0400
1
1---20141105'0500
1
1---20141105'0600
1
1---20141105'0700
1
1---20141105'0800
1
1---20141105'0900
1
1---20141105'1000
1
1---20141105'1100
1
1---20141105'1200
1
1---20141105'1300
1
1---20141105'1400
1
1---20141105'1500
1
1---20141105'1600
1
1---20141105'1700
1
1---20141105'1800
1
1---20141105'1900
1
1---20141105'2000
1
1---20141105'2100
1
1---20141105'2200
1
1---20141105'2300
1
1---20141106'0000
1
1---20141106'0100
1
1---20141106'0200
1
1---20141106'0300
1
1---20141106'0400
1
1---20141106'0500
1
1---20141106'0600
1
1---20141106'0700
1
1---20141106'0800
1
1---20141106'0900
1
1---20141106'1000
1
1---20141106'1100
1
1---20141106'1200
1
1---20141106'1300
1
1---20141106'1400
1
1---20141106'1500
1
1---20141106'1600
1
1---20141106'1700
1
1---20141106'1800
1
1---20141106'1900
1
3---20141106'2015
1---20141106'2100
1
1---20141106'2200
1
1---20141106'2300
1
1-

1---20141122'0300
1
1---20141122'0400
1
1---20141122'0500
1
1---20141122'0600
1
1---20141122'0700
1
1---20141122'0800
1
1---20141122'0900
1
1---20141122'1000
1
1---20141122'1100
1
1---20141122'1200
1
1---20141122'1300
1
1---20141122'1400
1
1---20141122'1500
1
1---20141122'1600
1
1---20141122'1700
1
1---20141122'1800
1
1---20141122'1900
1
1---20141122'2000
1
1---20141122'2100
1
1---20141122'2200
1
1---20141122'2300
1
1---20141123'0000
1
1---20141123'0100
1
1---20141123'0200
1
1---20141123'0300
1
1---20141123'0400
1
1---20141123'0500
1
1---20141123'0600
1
1---20141123'0700
1
1---20141123'0800
1
1---20141123'0900
1
1---20141123'1000
1
1---20141123'1100
1
1---20141123'1200
1
1---20141123'1300
1
1---20141123'1400
1
1---20141123'1500
1
1---20141123'1600
1
1---20141123'1700
1
1---20141123'1800
1
1---20141123'1900
1
1---20141123'2000
1
1---20141123'2100
1
1---20141123'2200
1
1---20141123'2300
1
1---20141124'0000
1
1---20141124'0100
1
1---20141124'0200
1
1---20141124'0300
1
1---20141124'0400
1


1---20141209'0600
1
1---20141209'0700
1
1---20141209'0800
1
1---20141209'0900
1
1---20141209'1000
1
1---20141209'1100
1
1---20141209'1200
1
1---20141209'1300
1
1---20141209'1400
1
1---20141209'1500
1
1---20141209'1600
1
1---20141209'1700
1
1---20141209'1800
1
1---20141209'1900
1
1---20141209'2000
1
1---20141209'2100
1
1---20141209'2200
1
1---20141209'2300
1
1---20141210'0000
1
1---20141210'0100
1
1---20141210'0200
1
1---20141210'0300
1
1---20141210'0400
1
1---20141210'0500
1
1---20141210'0600
1
1---20141210'0700
1
1---20141210'0800
1
1---20141210'0900
1
1---20141210'1000
1
1---20141210'1100
1
1---20141210'1200
1
1---20141210'1300
1
1---20141210'1400
1
1---20141210'1500
1
1---20141210'1600
1
1---20141210'1700
1
1---20141210'1800
1
1---20141210'1900
1
1---20141210'2000
1
1---20141210'2100
1
1---20141210'2200
1
1---20141210'2300
1
1---20141211'0000
1
1---20141211'0100
1
1---20141211'0200
1
1---20141211'0300
1
1---20141211'0400
1
1---20141211'0500
1
1---20141211'0600
1
1---20141211'0700
1


...

1---20150112'1500
1
1---20150112'1600
1
1---20150112'1700
1
1---20150112'1800
1
1---20150112'1900
1
1---20150112'2000
1
1---20150112'2100
1
1---20150112'2200
1
1---20150112'2300
1
1---20150113'0000
1
1---20150113'0100
1
1---20150113'0200
1
1---20150113'0300
1
1---20150113'0400
1
1---20150113'0500
1
1---20150113'0600
1
1---20150113'0700
1
1---20150113'0800
1
1---20150113'0900
1
1---20150113'1000
1
1---20150113'1100
1
1---20150113'1200
1
1---20150113'1300
1
1---20150113'1400
1
1---20150113'1500
1
1---20150113'1600
1
1---20150113'1700
1
1---20150113'1800
1
1---20150113'1900
1
1---20150113'2000
1
1---20150113'2100
1
1---20150113'2200
1
1---20150113'2300
1
1---20150114'0000
1
1---20150114'0100
1
1---20150114'0200
1
1---20150114'0300
1
1---20150114'0400
1
1---20150114'0500
1
1---20150114'0600
1
1---20150114'0700
1
1---20150114'0800
1
1---20150114'0900
1
1---20150114'1000
1
1---20150114'1100
1
1---20150114'1200
1
1---20150114'1300
1
1---20150114'1400
1
1---20150114'1500
1
1---20150114'1600
1


1---20150128'2200
1
1---20150128'2300
1
1---20150129'0000
1
1---20150129'0100
1
1---20150129'0200
1
1---20150129'0300
1
1---20150129'0400
1
1---20150129'0500
1
1---20150129'0600
1
1---20150129'0700
1
1---20150129'0800
1
1---20150129'0900
1
1---20150129'1000
1
1---20150129'1100
1
1---20150129'1200
1
1---20150129'1300
1
1---20150129'1400
1
1---20150129'1500
1
1---20150129'1600
1
1---20150129'1700
1
1---20150129'1800
1
1---20150129'1900
1
3---20150129'2015
1---20150129'2100
1
1---20150129'2200
1
1---20150129'2300
1
1---20150130'0000
1
1---20150130'0100
1
1---20150130'0200
1
1---20150130'0300
1
1---20150130'0400
1
1---20150130'0500
1
1---20150130'0600
1
1---20150130'0700
1
1---20150130'0800
1
1---20150130'0900
1
1---20150130'1000
1
1---20150130'1100
1
1---20150130'1200
1
1---20150130'1300
1
1---20150130'1400
1
1---20150130'1500
1
1---20150130'1600
1
1---20150130'1700
1
1---20150130'1800
1
1---20150130'1900
1
1---20150130'2000
1
1---20150130'2100
1
1---20150130'2200
1
1---20150130'2300
1
1-

1---20150215'0300
1
1---20150215'0400
1
1---20150215'0500
1
1---20150215'0600
1
1---20150215'0700
1
1---20150215'0800
1
1---20150215'0900
1
1---20150215'1000
1
1---20150215'1100
1
1---20150215'1200
1
1---20150215'1300
1
1---20150215'1400
1
1---20150215'1500
1
1---20150215'1600
1
1---20150215'1700
1
1---20150215'1800
1
1---20150215'1900
1
1---20150215'2000
1
1---20150215'2100
1
1---20150215'2200
1
1---20150215'2300
1
1---20150216'0000
1
1---20150216'0100
1
1---20150216'0200
1
1---20150216'0300
1
1---20150216'0400
1
1---20150216'0500
1
1---20150216'0600
1
1---20150216'0700
1
1---20150216'0800
1
1---20150216'0900
1
1---20150216'1000
1
1---20150216'1100
1
1---20150216'1200
1
1---20150216'1300
1
1---20150216'1400
1
1---20150216'1500
1
1---20150216'1600
1
1---20150216'1700
1
1---20150216'1800
1
1---20150216'1900
1
1---20150216'2000
1
1---20150216'2100
1
1---20150216'2200
1
1---20150216'2300
1
1---20150217'0000
1
1---20150217'0100
1
1---20150217'0200
1
1---20150217'0300
1
1---20150217'0400
1


1---20150304'0800
1
1---20150304'0900
1
1---20150304'1000
1
1---20150304'1100
1
1---20150304'1200
1
1---20150304'1300
1
1---20150304'1400
1
1---20150304'1500
1
1---20150304'1600
1
1---20150304'1700
1
1---20150304'1800
1
1---20150304'1900
1
1---20150304'2000
1
1---20150304'2100
1
1---20150304'2200
1
1---20150304'2300
1
1---20150305'0000
1
1---20150305'0100
1
1---20150305'0200
1
1---20150305'0300
1
1---20150305'0400
1
1---20150305'0500
1
1---20150305'0600
1
1---20150305'0700
1
1---20150305'0800
1
1---20150305'0900
1
1---20150305'1000
1
1---20150305'1100
1
1---20150305'1200
1
1---20150305'1300
1
1---20150305'1400
1
1---20150305'1500
1
1---20150305'1600
1
1---20150305'1700
1
1---20150305'1800
1
1---20150305'1900
1
3---20150305'2015
1---20150305'2100
1
1---20150305'2200
1
1---20150305'2300
1
1---20150306'0000
1
1---20150306'0100
1
1---20150306'0200
1
1---20150306'0300
1
1---20150306'0400
1
1---20150306'0500
1
1---20150306'0600
1
1---20150306'0700
1
1---20150306'0800
1
1---20150306'0900
1
1-

1---20150321'1400
1
1---20150321'1500
1
1---20150321'1600
1
1---20150321'1700
1
1---20150321'1800
1
1---20150321'1900
1
1---20150321'2000
1
1---20150321'2100
1
1---20150321'2200
1
1---20150321'2300
1
1---20150322'0000
1
1---20150322'0100
1
1---20150322'0200
1
1---20150322'0300
1
1---20150322'0400
1
1---20150322'0500
1
1---20150322'0600
1
1---20150322'0700
1
1---20150322'0800
1
1---20150322'0900
1
1---20150322'1000
1
1---20150322'1100
1
1---20150322'1200
1
1---20150322'1300
1
1---20150322'1400
1
1---20150322'1500
1
1---20150322'1600
1
1---20150322'1700
1
1---20150322'1800
1
1---20150322'1900
1
1---20150322'2000
1
1---20150322'2100
1
1---20150322'2200
1
1---20150322'2300
1
1---20150323'0000
1
1---20150323'0100
1
1---20150323'0200
1
1---20150323'0300
1
1---20150323'0400
1
1---20150323'0500
1
1---20150323'0600
1
1---20150323'0700
1
1---20150323'0800
1
1---20150323'0900
1
1---20150323'1000
1
1---20150323'1100
1
1---20150323'1200
1
1---20150323'1300
1
1---20150323'1400
1
1---20150323'1500
1


In [14]:
print(len(list_da))
print(len(list_ti))
print(len(list_m))
print(len(list_cx))
print(len(list_cy))
print(len(list_sx))
print(len(list_sy))
print(len(list_skx))
print(len(list_sky))
print(len(list_kx))
print(len(list_ky))
print(len(list_cloud_pixel_num))
print(len(list_non_nan_num))
print(len(list_percentage_cloud_cover))
print(len(list_len))
print(len(list_case))

4366
4366
4366
4366
4366
4366
4366
4366
4366
4366
4366
4366
4366
4366
4366
4366


### TO-DO: Change the directory for outputs

Please clearly label the result as it is coming from Step 4.

In [15]:
#columns = ['Filename', 'Mean', 'Std', 'Skewness']
data = {'Date':list_da, 'Time': list_ti,'Mean':list_m, 
        'Centroid_lon':list_cx,'Centroid_lat':list_cy,
        'Std_lon':list_sx,'Std_lat':list_sy,'Skewness_lon':list_skx, 'Skewness_lat':list_sky,
        'Kurtosis_lon': list_kx, 'Kurtosis_lat': list_ky,
        'Cloud_Cover_Count': list_cloud_pixel_num,
        'Total_Lake_Pixel': list_non_nan_num, 'Percent_Cloud_Cover': list_percentage_cloud_cover,
        'Sample Number': list_len, 'Selected':list_case}
binned = pd.DataFrame(data = data)

## TO-DO: Change output file name.
binned.to_csv("04_binned_lat_lon_opencv_14f15s_lake_0.csv",index = False)

In [16]:
binned

,Date,Time,Mean,Centroid_lon,Centroid_lat,Std_lon,Std_lat,Skewness_lon,Skewness_lat,Kurtosis_lon,Kurtosis_lat,Cloud_Cover_Count,Total_Lake_Pixel,Percent_Cloud_Cover,Sample Number,Selected
0,20141001,'0000,0.001928,-86.746619,43.883567,0.662943,1.224128,0.649933,0.080196,2.899997,1.760560,1364.0,2458.0,0.554923,2,s00
1,20141001,'0100,0.001935,-86.742887,43.944112,0.643303,1.220939,0.645693,0.036302,2.965771,1.725964,1331.0,2411.0,0.552053,4,s00
2,20141001,'0200,0.001960,-86.748614,43.942191,0.632510,1.227168,0.676957,0.002681,3.004695,1.741192,1278.0,2373.0,0.538559,4,s00
3,20141001,'0300,0.001980,-86.727879,43.950843,0.643921,1.217663,0.555008,0.031485,2.822773,1.688957,1281.0,2324.0,0.551205,3,s00
4,20141001,'0400,0.001894,-86.755259,43.912042,0.639526,1.217441,0.689993,0.026102,3.043969,1.729670,1270.0,2374.0,0.534962,4,s00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,20150331,'1900,0.424953,-86.452618,44.227281,0.634857,1.384043,0.424670,-0.303650,2.832995,1.487664,1990.0,3599.0,0.552931,4,s00
4362,20150331,'2000,0.483508,-86.392889,44.431771,0.642376,1.309728,0.133097,-0.637916,2.567251,1.903678,1844.0,3599.0,0.512365,3,s00
4363,20150331,'2100,0.476928,-86.429517,44.437452,0.687164,1.252320,0.112368,-0.644982,2.347145,2.093536,1800.0,3599.0,0.500139,3,s00
4364,20150331,'2200,0.389742,-86.505643,44.398012,0.699705,1.242613,0.215426,-0.534133,2.299630,1.938475,1888.0,3599.0,0.524590,4,s00
